코드 생성

In [197]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate
from mpl_toolkits.mplot3d import Axes3D
from collections import deque  
import random as rand

sats = [(1, 5), (2, 6), (3, 7), (4, 8), (0, 8), (1, 9), (0, 7), (1, 8), (2, 9), (1, 2),
            (2, 3), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (0, 3), (1, 4), (2, 5), (3, 6),
            (4, 7), (5, 8), (0, 2), (3, 5), (4, 6), (5, 7), (6, 8), (7, 9), (0, 5), (1, 6),
            (2, 7), (3, 8), (4, 9), (3, 9), (0, 6), (1, 7), (3, 9)]
g1tap = [2,9]
g2tap = [1,2,5,7,8,9]

def getCode(satsNum):
    
    g1 = deque(1 for i in range(10))
    g2 = deque(1 for i in range(10))
    
    # result
    g = []
    
    # Generating 1023 chips(One C/A sequence)
    for i in range(1023):
        val = (g1[9] + g2[satsNum[0]] + g2[satsNum[1]]) % 2
        g.append(val)
        
        #shift g1
        g1[9] = sum(g1[i] for i in g1tap) % 2
        g1.rotate()
        
        #shift g2
        g2[9] = sum(g2[i] for i in g2tap) % 2
        g2.rotate()
    # 0 => -1
    for n,i in enumerate(g):
            if i==0:
                g[n]=-1
        
    return g

# 코드 미리 만들어두기(시간복잡도 줄이려고)
codes = []
for i in range(37):
    codes.append(getCode(sats[i]))

In [198]:
OV = 1
ms = 100 # message 한 bit 마다 20ms

Data 생성

In [199]:
data = []
for i in range(ms//20):
    data.append((-1)**i)
RN = rand.randint(0,36)
t_code = codes[RN]
t_code = [chip for chip in t_code for _ in range(OV)]
t_code_inv = [x*-1 for x in t_code]
seq = []
for d in data:
    if d == 1:
        seq.extend(t_code*20)
    else:
        seq.extend(t_code_inv*20)



Signal 생성

In [200]:
def generate_transmitted_signal(code_delay, doppler_freq, svNumber, code_freq, signal_length, OV = 1.023e6*OV):
    delayed_seq = np.roll(seq, code_delay) # code_delay : delay된 sample 수
    
    t = np.arange(signal_length)
    code_phase = 2 * np.pi * code_freq * t / OV
    oversampled_signal = np.cos(code_phase) * np.exp(1j * (2 * np.pi * doppler_freq * t / OV))
    
    signal = delayed_seq * oversampled_signal[:signal_length]
    return signal
    

Replica 신호 생성

In [201]:
def generate_replica_signal(code_delay, doppler_freq, code_freq, svNumber, signal_length = 1023*OV ,OV = 1.023e6*OV ):
    ca_code = np.array(codes[svNumber])
    ca_code = [chip for chip in ca_code for _ in range(int(OV//1.023e6))]
    delayed_code = np.roll(ca_code, code_delay)
    
    t = np.arange(signal_length)
   
    code_phase = 2 * np.pi * code_freq * t / OV
    oversampled_signal = np.cos(code_phase) * np.exp(1j * (2 * np.pi * (doppler_freq) * t / OV))
    
    replica =  delayed_code * oversampled_signal[:signal_length]
    replica = np.tile(replica,20)
    return replica
    

Acquisition

In [202]:
def acquisition(signal, code_delay_range, doppler_freq_range,code_freq, svNumber, signal_length, OV):
    max_corr = 0
    max_code_delay = 0
    max_doppler_freq = 0
    max_cor_lst = []
    signal_div = [signal[i:i+(1023*20*OV)] for i in range(0,len(signal), 1023*20*OV)]
    
    for i, doppler_freq in enumerate(doppler_freq_range):
        replica = generate_replica_signal(code_delay_range[0],doppler_freq,code_freq, svNumber)
        
        for j, code_delay in enumerate(code_delay_range):
            correlation = []
            for lst in signal_div:
                cor = np.abs(correlate(lst, replica, mode='valid'))
                correlation.append(np.max(cor))
            avg_cor = sum(correlation)/len(signal_div)
            
            if avg_cor > max_corr:
                max_corr = avg_cor
                max_code_delay = code_delay
                max_doppler_freq = doppler_freq
                max_cor_lst = correlation
                
            replica = np.roll(replica, OV)
    return max_corr, max_code_delay, max_doppler_freq, max_cor_lst
            

Acquisition 실행

In [203]:
# Parameters
code_delay_range = np.arange(0, 200*OV, OV)  # Range of code delay in chips
doppler_freq_range = np.linspace(-5000, 5000, 21)  # Adjusted range of Doppler frequency in Hz
code_freq = 1.023e6  # Code frequency in Hz
svNumber = RN
# Generate received signal
true_code_delay = rand.randint(0,200*OV)
true_doppler_freq = rand.randint(-5000,5000)
received_signal = generate_transmitted_signal(true_code_delay, true_doppler_freq, svNumber, code_freq, 1023*20*(ms//20))

# Perform acquisition
max_corr, estimated_code_delay, estimated_doppler_freq, est_cor_lst= acquisition(received_signal, code_delay_range, doppler_freq_range, code_freq, svNumber,1023*20*(ms//20),OV)

print("-"*40)
print("Target SV Number:", svNumber + 1)
print("True Code Delay:", true_code_delay/OV)
print("True doppler frequency:", true_doppler_freq)
print("Maximum correlation:", max_corr)
print("Estimated code delay:", estimated_code_delay/OV)
print("Estimated Doppler frequency:", estimated_doppler_freq)
print('Est_cor_lst :',est_cor_lst)

----------------------------------------
Target SV Number: 37
True Code Delay: 77.0
True doppler frequency: -593
Maximum correlation: 449.86593998892494
Estimated code delay: 77.0
Estimated Doppler frequency: -500.0
Est_cor_lst : [382.8581948593738, 466.61787627131247, 466.6178762713126, 466.617876271313, 466.6178762713132]
